# Creating Spam Filter using multinomial Naive Bayes algorithm

Main goal of this project is to create a model, that based on near 6000 classified messages as SPAM or nonSPAM (labeled as HAM), it will distinguish if new message is SPAM or not. In this project I will be using multinomial Naive Bayes algorithm with some introduction to conditional probability.

In [1]:
import pandas as pd

In [2]:
messages = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label','SMS'])

In [18]:
messages.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


Lets check what is percentages of SPAM and HAM messages.

In [5]:
messages['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

In [6]:
# radnomize messages first
messagess_randomized = messages.sample(frac=1, random_state=1)

In [59]:
# divide data into train and test set - 80% as training set, 20% as a test set
train_len = round(len(messagess_randomized) * 0.8)
train_msg = messagess_randomized[:train_len].copy().reset_index(drop=True)
test_msg = messagess_randomized[train_len:].copy().reset_index(drop=True)

In [60]:
# check if train set has similar percentages of SPAM and HAM messages.
train_msg["Label"].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [61]:
# check if test set has similar percentages of SPAM and HAM messages.
test_msg["Label"].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

# Data cleaning

In [62]:
# replace all non-word characters by whitespace - only words are welcome here
test_msg['SMS'] = test_msg['SMS'].str.replace('\W', ' ')
test_msg['SMS'].head()

0            Later i guess  I needa do mcat study too 
1               But i haf enuff space got like 4 mb   
2    Had your mobile 10 mths  Update to latest Oran...
3    All sounds good  Fingers   Makes it difficult ...
4    All done  all handed in  Don t know if mega sh...
Name: SMS, dtype: object

In [63]:
# replace all non-word characters by whitespace - only words are welcome here
train_msg['SMS'] = train_msg['SMS'].str.replace('\W', ' ')
train_msg['SMS'].head()

0                         Yep  by the pretty sculpture
1        Yes  princess  Are you going to make me moan 
2                           Welp apparently he retired
3                                              Havent 
4    I forgot 2 ask ü all smth   There s a card on ...
Name: SMS, dtype: object

In [64]:
# converting strings to lowercase
test_msg['SMS'] = test_msg['SMS'].str.lower()

In [65]:
# converting strings to lowercase
train_msg['SMS'] = train_msg['SMS'].str.lower()

In [66]:
# now lets split all words in train text messages to list of words
train_msg['SMS'] = train_msg['SMS'].str.split()

In [67]:
train_msg['SMS'].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

# Some theory behind multinomial Naive Bayes algorithm (with help dataquest.io)

Using Bayes Theorem its possible to calculate probability that new message is a spam: <br>
<br>
$$ P(SPAM|new\;message)\;=\; \frac{P(SPAM) \cdot P(new\;message|SPAM)}{P(new\;message)} $$
$$ P(HAM|new\;message)\;=\; \frac{P(HAM) \cdot P(new\;message|HAM)}{P(new\;message)}  $$

Since I need to only compare the values, it is possible to ignore division:<br>
<br>
$$ P(SPAM|new\;message)\;\displaystyle \propto\; P(SPAM) \cdot P(new\;message|SPAM)$$
$$ P(HAM|new\;message)\;\displaystyle \propto\; P(HAM) \cdot P(new\;message|HAM) $$

Given simple table:

| LABEL | SMS                                            |
|-------|------------------------------------------------|
|   ?   | Secret code to unlock gift                     |

<br>
It is possible to calculate: <br>
<br>
$$ P(SPAM|"Secret\;code\;to\;unlockgift")\displaystyle \propto P(SPAM) \cdot P("Secret\;code\;to\;unlock\;gift"|SPAM) $$
$$ P(HAM|"Secret\;code\;to\;unlock\;gift")\;\displaystyle \propto\;P(HAM) \cdot P("Secret\;code\;to\;unlock\;gift"|HAM) $$
Also
$$ P(SPAM|w1,...,w5) = \frac{P(SPAM\cap(w1\;\cap...\cap\; w5)}{P(w1\;\cap\;...\;\cap\; w5)}$$
$$ P(SPAM|w1,...,w5)\;\displaystyle \propto\;P(SPAM\cap(w1\;\cap...\cap\; w5))$$

And here it is, Naive Bayes algorithm assumes that words (w1,w2,w3,w4,w5) are conditional independent. Which is not true, i.e. word "free" will occur more frequent with words like "prize" or "gift". Thus ignoring conditional dependece is called 'naive' in this situation. Simplifying above equation:

$$ P(SPAM|w1,...,w5)\;\displaystyle \propto\;P(SPAM)\cdot P(w1|SPAM)\;\cdot\;...\;\cdot\; P(w5|SPAM)$$
$$ P(HAM|w1,...,w5)\;\displaystyle \propto\;P(HAM)\cdot P(w1|HAM)\;\cdot\;...\;\cdot\; P(w5|HAM)$$

Now given below data set:

| LABEL | SMS                                                  |
|-------|------------------------------------------------------|
| SPAM  | WINNER!! You won a Secret gift!                      |
|  HAM  | Hey hun, please buy groceries when you get back      |
| SPAM  | Free prize! Just text us back with 12123 code.       |
|   ?   | Secret code to unlock gift                           |

I get probabilitities:
$$ N_{SPAM} = 15 $$
$$ N_{HAM} = 9 $$
$$ N_{VOCABULARY} = N_{SPAM} + N_{HAM} = 24 $$
$$ \alpha = 1 <--- additive\;smoothing $$
NOTE: for more info about additive smoothing check out https://en.wikipedia.org/wiki/Additive_smoothing
___________________________________________________________________________

$$ P(SPAM) = \frac{2}{3} $$
$$ P("Secret"|SPAM) = \frac{N_{"Secret"|SPAM} + \alpha}{N_{SPAM} + N_{VOCABULARY}} = \frac{1 + 1}{15 + 24} = \frac{2}{39}$$
$$ P("code"|SPAM) = \frac{2}{39}$$
$$ P("to"|SPAM) = \frac{1}{39}$$
$$ P("unlock"|SPAM) = \frac{1}{39}$$
$$ P("gift"|SPAM) = \frac{2}{39}$$
$$ P(SPAM|"Secret\;code\;to\;unlockgift")\displaystyle \propto \frac{2}{3} \cdot \frac{2}{39} \cdot \frac{2}{39} \cdot \frac{1}{39} \cdot \frac{1}{39} \cdot \frac{2}{39} = \frac{16}{270672597} = 5,9e-8$$

___________________________________________________________________________
$$ P(HAM) = \frac{1}{3} $$
$$ P("Secret"|HAM) = \frac{N_{"Secret"|SPAM} + \alpha}{N_{HAM} + N_{VOCABULARY}} = \frac{0 + 1}{9 + 24} = \frac{1}{33}$$
$$ P("code"|HAM) = \frac{1}{33}$$
$$ P("to"|HAM) = \frac{1}{33}$$
$$ P("unlock"|HAM) = \frac{1}{33}$$
$$ P("gift"|HAM) = \frac{1}{33}$$
$$ P(HAM|"Secret\;code\;to\;unlockgift")\displaystyle \propto \frac{1}{3} \cdot \frac{1}{33} \cdot \frac{1}{33} \cdot \frac{1}{33} \cdot \frac{1}{33} \cdot \frac{1}{33} = \frac{1}{39135393} = 2,56e-8$$
___________________________________________________________________________
Now I can conclude that message is a SPAM
$$ P(SPAM|"Secret\;code\;to\;unlockgift") > P(HAM|"Secret\;code\;to\;unlockgift") $$


# Preparing data for multinomial Naive Bayes algorithm

Presented algorithm I will apply to my model. First lets extract vocabulary from all messages.

In [68]:
vocabulary = []
for sms in train_msg['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

In [69]:
print(len(vocabulary))

7783


In [70]:
# initialize dictionary that counts occurence of SMS words in vocabulary
word_counts_per_sms = {unique_word : [0] * len(train_msg['SMS']) for unique_word in vocabulary}

In [71]:
train_msg.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [72]:
for index, sms in enumerate(train_msg['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [73]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,invited,kidding,145,700,messenger,mths,stays,txt,complain,accounts,...,honi,constantly,6zf,celeb,fancies,arm,mtmsg18,fo,jog,83600
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# concatenate word counts data frame to training set
clean_train_msg = pd.concat([train_msg, word_counts], axis=1)

In [76]:
clean_train_msg.head()

,Label,SMS,invited,kidding,145,700,messenger,mths,stays,txt,...,honi,constantly,6zf,celeb,fancies,arm,mtmsg18,fo,jog,83600
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
p_spam = clean_train_msg['Label'].value_counts(normalize=True)['spam']
print('P(SPAM) = {}'.format(p_spam))

P(SPAM) = 0.13458950201884254


In [81]:
p_ham = clean_train_msg['Label'].value_counts(normalize=True)['ham']
print('P(HAM) = {}'.format(p_ham))

P(HAM) = 0.8654104979811574


In [82]:
n_spam_words = clean_train_msg[clean_train_msg['Label'] == 'spam']['SMS'].apply(len).sum()
print('N_SPAM = {}'.format(n_spam_words))

N_SPAM = 15190


In [83]:
n_ham_words = clean_train_msg[clean_train_msg['Label'] == 'ham']['SMS'].apply(len).sum()
print('N_HAM = {}'.format(n_ham_words))

N_HAM = 57237


In [84]:
n_vocabulary = len(vocabulary)
print('N_VOCABULARY = {}'.format(n_vocabulary))

N_VOCABULARY = 7783


In [85]:
alpha = 1

In [86]:
# initialize dictionary of P(w|SPAM) and dictionary P(w|HAM)
p_word_given_spam = {unique_word: 0 for unique_word in vocabulary}
p_word_given_ham = {unique_word: 0 for unique_word in vocabulary}

In [87]:
clean_train_set_spam = clean_train_msg[clean_train_msg['Label'] == 'spam']
clean_train_set_ham = clean_train_msg[clean_train_msg['Label'] == 'ham']

In [88]:
# initialize N_word|SPAM and N_word|HAM
n_word_given_spam = {unique_word: 0 for unique_word in vocabulary}
n_word_given_ham = {unique_word: 0 for unique_word in vocabulary}

In [89]:
# count N_word|SPAM and N_word|HAM
for unique_word in vocabulary:
    n_word_given_spam[unique_word] = clean_train_set_spam[unique_word].sum()
    n_word_given_ham[unique_word] = clean_train_set_ham[unique_word].sum()

In [90]:
# count P(w|SPAM) and P(w|HAM)
for unique_word in vocabulary:
    p_word_given_spam[unique_word] = (n_word_given_spam[unique_word] + alpha)/(n_spam_words + alpha*n_vocabulary)
    p_word_given_ham[unique_word] = (n_word_given_ham[unique_word] + alpha)/(n_ham_words + alpha*n_vocabulary)

All needed probablilities were calculated. Now lets create function that classifies messages to SPAM or HAM.

In [110]:
import re

def classify(message):

    # clean message
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:

    p_spam_given_message = ?
    p_ham_given_message = ?
    '''    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in vocabulary:
            p_spam_given_message *= p_word_given_spam[word]
            p_ham_given_message *= p_word_given_ham[word]

    #print('P(Spam|message):', p_spam_given_message)
    #print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'This message needs human judgement'

In [111]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

'spam'

In [112]:
classify("Sounds good, Tom, then see u there")

'ham'

# Test model

In [113]:
test_msg['predicted'] = test_msg['SMS'].apply(classify)

In [114]:
test_msg.head()

,Label,SMS,predicted
0,ham,later i guess i needa do mcat study too,ham
1,ham,but i haf enuff space got like 4 mb,ham
2,spam,had your mobile 10 mths update to latest oran...,spam
3,ham,all sounds good fingers makes it difficult ...,ham
4,ham,all done all handed in don t know if mega sh...,ham


In [115]:
correct_prediction = 0
total = len(test_msg['SMS'])

In [116]:
for row in test_msg.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct_prediction += 1

In [117]:
print('Correct prediction:', correct_prediction)
print('Incorrect prediction:', total - correct_prediction)
print('Accuracy:', correct_prediction/total)

Correct prediction: 1100
Incorrect prediction: 14
Accuracy: 0.9874326750448833


It seems that on 1114 messages, model failed to classify 14 messages, which gives 98,74% accuracy. Now, lets check out messages that was incorrecty classified.

In [124]:
for row in test_msg.iterrows():
    row = row[1]
    if row['Label'] != row['predicted']:
        print('___________________________________________________')
        print('Message labeled: {}, Message label predicted: {}'.format(row['Label'], row['predicted']))
        print(row['SMS'])
        print('\n')

___________________________________________________
Message labeled: spam, Message label predicted: ham
not heard from u4 a while  call me now am here all night with just my knickers on  make me beg for it like u did last time 01223585236 xx luv nikiyu4 net


___________________________________________________
Message labeled: spam, Message label predicted: ham
more people are dogging in your area now  call 09090204448 and join like minded guys  why not arrange 1 yourself  there s 1 this evening  a 1 50 minapn ls278bb


___________________________________________________
Message labeled: ham, Message label predicted: spam
unlimited texts  limited minutes 


___________________________________________________
Message labeled: ham, Message label predicted: spam
26th of july


___________________________________________________
Message labeled: ham, Message label predicted: spam
nokia phone is lovly  


___________________________________________________
Message labeled: ham, Message labe

Probably model had some problem with SMS that has unique words like "ni8", "ta". Also in those messages were one letters 'r', 'u', 't'. Alone Letter like 't'  was caused by replacing "'" with space i.e don"t -> don t. For better results I could get rid of those letters, and words like "and", "or", "dont", because those words has less information.